In [1]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import requests
import warnings
from time import sleep
import urllib.request
warnings.filterwarnings("ignore")
import random

In [2]:
#קבלת עמוד שלם
def getHtmlPage(url):
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    return soup

### Extract animals data from https://www.komo.co.il

At this part we convert the main html pages into BeautifulSoup format.    We took the ad numbers from each page to get a list of all the vehicle ads.

In [3]:
#קבלת כל מספרי הזיהוי של היצרנים
def list_of_url():
    soup=getHtmlPage('https://www.komo.co.il/code/cars/quick-links.asp?')
    tag = soup.find("div", attrs = {"class":"CardStyle"})
    producers = [i['href'] for i in tag.findAll("a")]
    for i in range(len(producers)):
        producers[i]=producers[i].replace("/code/cars/private.asp?yazranNumH=", "")
        
    return producers

This function turns the whole page into a list of links, the whole link is one car ad.

In [4]:
#הפיכת העמוד השלם לרשימה של קישורים, כל קישור הוא מודעת רכב אחת
def getListCar(soup):
    car_IDs = [] 
    cars=soup.select("div[id^=modaaRowDv]")
    for car in cars:
        car_ID =car.find("div")["id"].replace("ModaaFavN", "")
        car_IDs.append(car_ID)    
    return car_IDs

 This function preparation of manufacturers' lists and regions.

In [16]:
#1. הכנת רשימות של יצרנים ואיזורים
area_IDs=[31,21,14,9,17,2,4,28,27,12,37,29,3,19,26,39,25,42,7,8,13,16,38,40,32,11,18,33,15,36,44,34,6,5,30,43] 
producer_IDs=list_of_url()#קבלת כל מספרי הזיהוי של היצרנים
print(producer_IDs)
#producer_IDs=[22]
#area_IDs=[31]
#print(producer_IDs)

['22', '31', '24', '43', '44', '52', '35', '56', '20', '40', '39', '12', '55', '49', '46', '2', '38', '41', '6', '34', '36', '48', '21', '371', '30', '19', '33', '53', '16', '13', '7', '51', '47', '23', '461', '362', '17', '1', '28', '45', '32', '42', '18', '54', '37', '26', '15', '11', '3']


This function gets a list with all the ads of the vehicles.

In [ ]:
#2. קבלת רשימה עם כל המודעות של הרכבים
car_IDs=[]

for a in area_IDs:#ריצה לפי איזור
    for p in producer_IDs:#ריצה לפי יצרן
        i=1
        new_car_IDs=[]
        while (len(new_car_IDs)!=0 or i==1) and i<=10:#ריצה על כל העמודים הלא ריקים ליצרן ואזור מסויים
            url="https://www.komo.co.il/code/cars/private.asp?currPage="+str(i)+"&subLuachNum=1&yazranNumH="+str(p)+"&ezorNum="+str(a)
            try:
                soup=getHtmlPage(url)
                new_car_IDs=getListCar(soup)#קבלת כל מספרי המודעה שיש בעמוד
                car_IDs=car_IDs+new_car_IDs#הוספת המודעות לרשימת המודעות שלנו
            except:
                new_car_IDs=[]
                print('error with url '+url)
                
            print(url+" new: "+str(len(new_car_IDs))+" total: "+str(len(car_IDs)) )
            i=i+1
            #sleep(random.randint(5, 10))



In [ ]:
df=pd.DataFrame({'modaa':car_IDs})
df.to_excel('carsIdS.xlsx')

Insert all ads into arrays.


In [ ]:
#3 הכנסת כל המודעות למערכים
car_price = []
car_area = []
car_producer = []
car_sub_type = []
car_year = []
car_km = []
car_yad = []
car_manoa = []
car_gir = []
car_manoa_type = []
car_color = []
car_baalot = []
car_ac = []
car_windows = []
car_sunroof = []
car_gps = []
car_air_bag = []

for c in range(0,len(car_IDs)):
    print(c)
    car_ID=car_IDs[c]
    car_url="https://www.komo.co.il/code/cars/details/?modaaNum="+car_ID
    car_soup=getHtmlPage(car_url)
    ##########################################
    ################מחיר#######################
    try:
        price = car_soup.select("span[class^=ModaaWDetailsValue]")[0].text.replace("₪","").replace("\xa0"," ").replace("\n","").strip()
    except:
        price="None"
    car_price.append(price)

    ##########################################
    ################איזור####################
    try:
        area = car_soup.select("div[class^=addresBottom ]")[0].text.replace("₪","").replace("\xa0"," ").replace("\n","").strip()
    except:
        area="None"
    car_area.append(area)

    ##########################################
    ################סוג ויצרן####################
    try:
        producer = car_soup.select("h1[class^=modaaWTitleNew]")[0].text.replace("₪","").replace("\xa0"," ").replace("\n","").strip()
        producer=producer[producer.find(','):].replace(",","")
    except:
        producer="None"
    car_producer.append(producer)

    ##########################################
    ################תת סוג#####################
    try:
        sub_type = car_soup.select("div[class^=md_f_15]")[0].text.replace("₪","").replace("\xa0"," ").replace("\n","").strip()
    except:
        sub_type="None"
    car_sub_type.append(sub_type)

    ##########################################
    ################שנה#######################
    try:
        year = car_soup.select("div[class^=year]")[0].text.replace("₪","").replace("\xa0"," ").replace("\n","").replace("שנה","").strip()
    except:
        year="None"
    car_year.append(year)

    ##########################################
    ################ק"מ#######################
    try:
        km = car_soup.select("div[class^=km]")[0].text.replace("₪","").replace("\xa0"," ").replace("\n","").replace('ק"מ',"").strip()
    except:
        km="None"
    car_km.append(km)

    ##########################################
    ################יד#########################
    try:
        yad = car_soup.select("div[class^=yad]")[0].text.replace("₪","").replace("\xa0"," ").replace("\n","").replace('יד',"").strip()
    except:
        yad="None"
    car_yad.append(yad)

    ##########################################
    ################מנוע#######################
    try:
        manoa = car_soup.select("div[class^=manoa ]")[0].text.replace("₪","").replace("\xa0"," ").replace("\n","").replace('סמ"ק',"").strip()
    except:
        manoa="None"
    car_manoa.append(manoa)

    ##########################################
    ################פרמטרים נוספים################
    try:
        modaaWNewUl  = car_soup.select("ul[class^=modaaWNewUl]")[0]
        modaaWNewUl = modaaWNewUl.select('li')
    except:
        modaaWNewUl="None"

    #עבור חלונות, גג נפתח, מזגן
    try:
        thirdInfo1  = car_soup.select("div[class^=thirdInfo1]")[0]
        thirdInfo1 = thirdInfo1.select('li')
    except:
        thirdInfo1="None"
    #GPS וכרית אויר ן
    try:
        thirdInfo2  = car_soup.select("div[class^=thirdInfo2]")[0]
        thirdInfo2 = thirdInfo2.select('li')
    except:
        thirdInfo2="None"

    ##########################################
    ################תיבת הילוכים#################
    try:
        gir  = modaaWNewUl[0].text.replace("₪","").replace("\xa0"," ").replace("\n","").replace('תיבת הילוכים:',"").strip()
    except:
        gir="None"
    car_gir.append(gir)

    ##########################################
    ################סוג מנוע####################
    try:
        manoa_type  = modaaWNewUl[1].text.replace("₪","").replace("\xa0"," ").replace("\n","").replace('סוג מנוע:',"").strip()
    except:
        manoa_type="None"
    car_manoa_type.append(manoa_type)

    ##########################################
    ################צבע########################
    try:
        color  = modaaWNewUl[2].text.replace("₪","").replace("\xa0"," ").replace("\n","").replace('צבע:',"").strip()
    except:
        color="None"
    car_color.append(color)

    ##########################################
    ################בעלות######################
    try:
        baalot  = modaaWNewUl[3].text.replace("₪","").replace("\xa0"," ").replace("\n","").replace('בעלות נוכחית:',"").strip()
    except:
        baalot="None"
    car_baalot.append(baalot)

    ##########################################
    ################מזגן########################
    try:
        if 'add' in thirdInfo1[0]['class']:
            ac  = 1 
        else:
            ac  = 0
    except:
        ac="None"
    car_ac.append(ac)

    ##########################################
    ################חלונות######################
    try:
        if 'add' in thirdInfo1[1]['class']:
            windows  = 1 
        else:
            windows  = 0
    except:
        windows="None"
    car_windows.append(windows)

    ##########################################
    ################גג נפתח#####################
    try:
        if 'add' in thirdInfo1[2]['class']:
            sunroof  = 1 
        else:
            sunroof  = 0
    except:
        sunroof="None"
    car_sunroof.append(sunroof)

    ##########################################
    ################GPS#######################
    try:
        if 'add' in thirdInfo2[0]['class']:
            gps  = 1 
        else:
            gps  = 0
    except:
        gps="None"
    car_gps.append(gps)

    ##########################################
    ################כרית אויר####################
    try:
        if 'add' in thirdInfo2[1]['class']:
            air_bag  = 1 
        else:
            air_bag  = 0
    except:
        air_bag="None"
    car_air_bag.append(air_bag)
#print(car_price)
#print(car_area)
#print(car_producer)
#print(car_sub_type)
#print(car_year)
#print(car_km)
#print(car_yad)
#print(car_manoa)
#print(car_gir)
#print(car_manoa_type)
#print(car_color)
#print(car_baalot)
#print(car_ac)
#print(car_windows)
#print(car_sunroof)
#print(car_gps)
#print(car_air_bag)

This function preparation of a data table from all the arrays we prepared above.

In [ ]:
#4 הכנת טבלת נתונים מכל המערכים שהכנו למעלה
df = pd.DataFrame({'modaa':car_IDs,
                   'price':car_price, 
                   'area':car_area,
                   'producer': car_producer, 
                   'sub_type': car_sub_type, 
                   'year' : car_year, 
                   'km': car_km, 
                   'yad': car_yad,  
                   'manoa': car_manoa,  
                   'gir': car_gir,  
                   'manoa_type': car_manoa_type,  
                   'color': car_color,  
                   'baalot': car_baalot,  
                   'ac': car_ac,  
                   'windows': car_windows,  
                   'sunroof': car_sunroof,  
                   'gps': car_gps,  
                   'air_bag': car_air_bag})
df.to_excel("cars_crawling.xlsx")
print(len(car_price))